In [ ]:
#import library
%pip install py_stringmatching

In [2]:
import py_stringmatching as sm
import pandas as pd

In [3]:
hospital_accounts = pd.read_csv('../data/hospital_account_info.csv')
hospital_reimbursement = pd.read_csv('../data/hospital_reimbursement.csv')

In [4]:
accountNameList = hospital_accounts['Facility Name'].tolist()
reimbursementNameList = hospital_reimbursement['Provider Name'].tolist()
accountNameTokens = []
reimbursementNameTokens = []
ws_tok = sm.WhitespaceTokenizer()
for name in accountNameList:
    accountNameTokens.append(ws_tok.tokenize(name))
for name in reimbursementNameList:
    reimbursementNameTokens.append(ws_tok.tokenize(name))
nameTokens = accountNameTokens + reimbursementNameTokens


In [5]:
soft_tfidf = sm.SoftTfIdf(nameTokens)
print(soft_tfidf.get_raw_score(['PEACEHEALTH', 'PEACE', 'ISLAND', 'MEDICAL', 'CENTER'],
    ['PEACHEALTH', 'ST', 'JOHN', 'MEDICAL', 'CENTER']))

1.1976107691264521


In [10]:
soft_tfidf = sm.SoftTfIdf(nameTokens, sim_func=sm.Jaro().get_raw_score, threshold=0.8)
rows = []
for i in range (0, len(accountNameTokens)):
    for j in range(0, len(reimbursementNameTokens)):
        rows.append([hospital_accounts.at[i,'Account_Num'], hospital_reimbursement.at[j, 'Provider_Num'], 
        soft_tfidf.get_raw_score(accountNameTokens[i], reimbursementNameTokens[j]), hospital_accounts.at[i,'Facility Name'], hospital_reimbursement.at[j, 'Provider Name'],
        hospital_accounts.at[i,'Address'], hospital_reimbursement.at[j, 'Provider Street Address']])

In [11]:
joinTable = pd.DataFrame(rows, columns =['Account_Num', 'Provider_Num', 'Similar Score', 'Facility Name', 'Provider Name', 'Address', 'Provider Street Address'])

In [12]:
#Sắp xếp điểm giảm dần
joinTable.sort_values(by='Similar Score', ascending=False, inplace=True)
joinTable.head(20)

,Account_Num,Provider_Num,Similar Score,Facility Name,Provider Name,Address,Provider Street Address
7130727,53716,498827,1.197449,PEACEHEALTH PEACE ISLAND MEDICAL CENTER,PEACHEALTH ST JOHN MEDICAL CENTER,1117 SPRING STREET,1615 DELAWARE STREET
7130730,53716,236029,1.173287,PEACEHEALTH PEACE ISLAND MEDICAL CENTER,PEACEHEALTH SOUTHWEST MEDICAL CENTER,1117 SPRING STREET,400 NE MOTHER JOSEPH PLACE
3054026,76102,750633,1.112292,"OUR LADY OF LOURDES MEMORIAL HOSPITAL, INC",LOURDES HOSPITAL,169 RIVERSIDE DRIVE,1530 LONE OAK ROAD
3960218,18074,750633,1.098185,"OUR LADY OF LOURDES REGIONAL MEDICAL CENTER, INC",LOURDES HOSPITAL,4801 AMBASSADOR CAFFERY PARKWAY,1530 LONE OAK ROAD
6681411,74526,718806,1.080431,ST LUKES LAKESIDE HOSPITAL,LAKES REGIONAL HEALTHCARE,17400 ST LUKES WAY,2301 HIGHWAY 71
602469,30486,496141,1.065649,OCEANS BEHAVIORAL HOSPITAL OF GREATER NEW ORLEANS,UNIVERSITY MEDICAL CENTER NEW ORLEANS,716 VILLAGE ROAD,2000 CANAL STREET
7523808,29583,775552,1.051866,BAYLOR SCOTT & WHITE TEXAS SPINE & JOINT HOSPITAL,SOUTH POINTE HOSPITAL,1814 ROSELAND BOULEVARD,20000 HARVARD ROAD
7495880,39225,623892,1.050294,TRINITY MEDICAL CTR EAST &TRINITY MEDICAL CTR ...,TRINITY REGIONAL MEDICAL CENTER,380 SUMMIT AVENUE,802 KENYON RD
7495521,39225,636204,1.048945,TRINITY MEDICAL CTR EAST &TRINITY MEDICAL CTR ...,MEDICAL CENTER OF TRINITY,380 SUMMIT AVENUE,"9330 SR 54, STE 401"
3054067,76102,345826,1.046712,"OUR LADY OF LOURDES MEMORIAL HOSPITAL, INC",OUR LADY OF THE LAKE REGIONAL MEDICAL CENTER,169 RIVERSIDE DRIVE,5000 HENNESSY BLVD


In [13]:
jointDistinctTable = joinTable[joinTable['Facility Name'] != joinTable['Provider Name']]
jointDistinctTable.head(50)

,Account_Num,Provider_Num,Similar Score,Facility Name,Provider Name,Address,Provider Street Address
7130727,53716,498827,1.197449,PEACEHEALTH PEACE ISLAND MEDICAL CENTER,PEACHEALTH ST JOHN MEDICAL CENTER,1117 SPRING STREET,1615 DELAWARE STREET
7130730,53716,236029,1.173287,PEACEHEALTH PEACE ISLAND MEDICAL CENTER,PEACEHEALTH SOUTHWEST MEDICAL CENTER,1117 SPRING STREET,400 NE MOTHER JOSEPH PLACE
3054026,76102,750633,1.112292,"OUR LADY OF LOURDES MEMORIAL HOSPITAL, INC",LOURDES HOSPITAL,169 RIVERSIDE DRIVE,1530 LONE OAK ROAD
3960218,18074,750633,1.098185,"OUR LADY OF LOURDES REGIONAL MEDICAL CENTER, INC",LOURDES HOSPITAL,4801 AMBASSADOR CAFFERY PARKWAY,1530 LONE OAK ROAD
6681411,74526,718806,1.080431,ST LUKES LAKESIDE HOSPITAL,LAKES REGIONAL HEALTHCARE,17400 ST LUKES WAY,2301 HIGHWAY 71
602469,30486,496141,1.065649,OCEANS BEHAVIORAL HOSPITAL OF GREATER NEW ORLEANS,UNIVERSITY MEDICAL CENTER NEW ORLEANS,716 VILLAGE ROAD,2000 CANAL STREET
7523808,29583,775552,1.051866,BAYLOR SCOTT & WHITE TEXAS SPINE & JOINT HOSPITAL,SOUTH POINTE HOSPITAL,1814 ROSELAND BOULEVARD,20000 HARVARD ROAD
7495880,39225,623892,1.050294,TRINITY MEDICAL CTR EAST &TRINITY MEDICAL CTR ...,TRINITY REGIONAL MEDICAL CENTER,380 SUMMIT AVENUE,802 KENYON RD
7495521,39225,636204,1.048945,TRINITY MEDICAL CTR EAST &TRINITY MEDICAL CTR ...,MEDICAL CENTER OF TRINITY,380 SUMMIT AVENUE,"9330 SR 54, STE 401"
3054067,76102,345826,1.046712,"OUR LADY OF LOURDES MEMORIAL HOSPITAL, INC",OUR LADY OF THE LAKE REGIONAL MEDICAL CENTER,169 RIVERSIDE DRIVE,5000 HENNESSY BLVD


In [14]:
jointDistinctTable.to_csv('softIF-IDF.csv')